In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from dave_ledger.core import paths, config

# Load Config
cfg = config.load_config()
current_year = cfg['context']['current_year']
history_years = cfg['context']['history_years']
years = [current_year - i for i in range(history_years)]

print(f"🎯 Targeting Years: {years}")
print(f"📂 Root Path: {paths.find_repo_root()}")

🎯 Targeting Years: [2025, 2024, 2023, 2022, 2021]
📂 Root Path: /home/winstonunderwood/projects/dave-ledger


In [2]:
from dave_ledger.etl import extract

In [5]:
extract.extract_xfp_data()

   -> Downloading xFP for 2025...
   -> Downloading xFP for 2024...
   -> Downloading xFP for 2023...
   -> Downloading xFP for 2022...
   -> Downloading xFP for 2021...
   -> ✅ Saved 29,617 rows of xFP data to xfp_2021_2025.parquet


In [8]:
# Cell 3: The Corrected Multi-Year Extract
import pandas as pd
import time

# 1. Define the URL Pattern
base_url = "https://github.com/ffverse/ffopportunity/releases/download/latest-data/ep_weekly_{}.parquet"
raw_dir = paths.find_repo_root() / "data" / "raw"
output_path = raw_dir / f"xfp_{min(years)}_{max(years)}.parquet"

all_years_data = []

print(f"📥 Starting download for years: {years}")

for year in years:
    url = base_url.format(year)
    print(f"   -> Downloading {year}...", end=" ")
    
    try:
        # Read directly from the year-specific URL
        df_year = pd.read_parquet(url)
        all_years_data.append(df_year)
        print("✅ Success")
        
    except Exception as e:
        print(f"❌ Failed: {e}")
        # Optional: break or continue depending on how strict you want to be
        # continue 

# 2. Combine all years
if all_years_data:
    df_xfp_raw = pd.concat(all_years_data, ignore_index=True)
    
    # Save the master file locally
    df_xfp_raw.to_parquet(output_path)
    
    print(f"\n🎉 Total Rows Downloaded: {len(df_xfp)}")
    print(f"💾 Saved to: {output_path}")
    
    # Show a sample to confirm it looks right
    display(df_xfp_raw.sample(3))
else:
    print("⚠️ No data was downloaded.")

📥 Starting download for years: [2025, 2024, 2023, 2022, 2021]
   -> Downloading 2025... ✅ Success
   -> Downloading 2024... ✅ Success
   -> Downloading 2023... ✅ Success
   -> Downloading 2022... ✅ Success
   -> Downloading 2021... ✅ Success

🎉 Total Rows Downloaded: 29617
💾 Saved to: /home/winstonunderwood/projects/dave-ledger/data/raw/xfp_2021_2025.parquet


,season,posteam,week,game_id,player_id,full_name,position,pass_attempt,rec_attempt,rush_attempt,...,total_yards_gained_diff_team,total_touchdown_team,total_touchdown_exp_team,total_touchdown_diff_team,total_first_down_team,total_first_down_exp_team,total_first_down_diff_team,total_fantasy_points_team,total_fantasy_points_exp_team,total_fantasy_points_diff_team
2110,2025,CAR,8.0,2025_08_BUF_CAR,00-0027973,Andy Dalton,QB,24.0,0.0,2.0,...,-15.48,1.0,1.21,-0.21,16.0,14.79,1.21,48.9,53.39,-4.49
8863,2024,SF,12.0,2024_12_SF_GB,None,None,None,0.0,3.0,0.0,...,-33.69,1.0,1.42,-0.42,11.0,14.52,-3.52,45.3,55.64,-10.34
11182,2024,LAC,9.0,2024_09_LAC_CLE,00-0039391,Kimani Vidal,RB,0.0,0.0,5.0,...,54.27,4.0,0.79,3.21,13.0,12.74,0.26,78.5,52.11,26.39


In [9]:
# 1. Check Column Names
# We look for any column containing 'fantasy' and 'exp' to find our target
cols = [c for c in df_xfp_raw.columns if 'fantasy' in c and 'exp' in c]
print(f"🔎 Found Potential xFP Columns: {cols}")
# 1. Define the columns we need
# We need the Total (Base) AND the Passing TD component (Adjustment)
target_cols = ['player_id', 'season', 'week', 'total_fantasy_points_exp', 'pass_touchdown_exp']

# 2. Extract and Rename
xfp_patch = df_xfp_raw[target_cols].copy()
xfp_patch = xfp_patch.rename(columns={'total_fantasy_points_exp': 'expected_fantasy_points'})

# Safety: Ensure IDs are strings
xfp_patch['player_id'] = xfp_patch['player_id'].astype(str)
df_main['player_id'] = df_main['player_id'].astype(str)

print(f"✅ Extracted {len(xfp_patch):,} rows of xFP data (with Pass TD ingredients).")
display(xfp_patch.head(3))
# 2. Look at a specific player to verify data makes sense
# Let's check a known player like Justin Jefferson (GSIS ID usually starts with '00-0036')
# Or just search by name if available (though nflverse usually has 'full_name' or 'player_name')
sample_player = df_xfp_raw[df_xfp_raw['full_name'] == 'Justin Jefferson'].sort_values(['season', 'week'])

if not sample_player.empty:
    display(sample_player[['season', 'week', 'total_fantasy_points_exp', 'total_fantasy_points']].tail(5))
else:
    print("⚠️ Could not find Justin Jefferson. Showing random sample:")
    display(df_xfp_raw[['season', 'week', 'total_fantasy_points_exp']].sample(5))

🔎 Found Potential xFP Columns: ['pass_fantasy_points_exp', 'rec_fantasy_points_exp', 'rush_fantasy_points_exp', 'total_fantasy_points_exp', 'pass_fantasy_points_exp_team', 'rec_fantasy_points_exp_team', 'rush_fantasy_points_exp_team', 'total_fantasy_points_exp_team']


,season,week,total_fantasy_points_exp,total_fantasy_points
3732,2025,13.0,8.58,2.4
4071,2025,14.0,7.39,3.1
4306,2025,15.0,18.82,4.2
4574,2025,16.0,11.10,14.5
4819,2025,17.0,9.54,7.0


In [11]:
# 1. Define the ingredients we need from the raw file
# We need the Base Total (4pt passing) and the Passing TD component
target_cols = ['player_id', 'season', 'week', 'total_fantasy_points_exp', 'pass_touchdown_exp']

# 2. Create the working dataframe
# We copy to avoid SettingWithCopy warnings on the raw slice
df_xfp_custom = df_xfp_raw[target_cols].copy()

# 3. Clean and Fill
# Fill NaNs with 0.0 immediately so math works
df_xfp_custom = df_xfp_custom.fillna(0.0)

# 4. Apply the Patch
# Formula: Base (4pt) + (Exp Pass TDs * 2 Extra Points)
df_xfp_custom['expected_fantasy_points'] = (
    df_xfp_custom['total_fantasy_points_exp'] + 
    (df_xfp_custom['pass_touchdown_exp'] * 2.0)
)

# 5. Final Cleanup
# We only keep the ID, time, and the final calculated value
final_xfp_df = df_xfp_custom[['player_id', 'season', 'week', 'expected_fantasy_points']].copy()

# Safety: Ensure IDs are strings for the eventual merge
final_xfp_df['player_id'] = final_xfp_df['player_id'].astype(str)

print(f"✅ Computed Custom xFP for {len(final_xfp_df):,} rows.")
display(final_xfp_df.head(5))

# Sanity Check: View a high-volume QB to ensure points look higher than standard
# (Josh Allen is usually around 20-25 xFP in standard, should be higher here)
sample_qb = df_xfp_raw[df_xfp_raw['full_name'] == 'Josh Allen'].iloc[0]
print(f"\nSample Check (Josh Allen - Week {sample_qb['week']}, {sample_qb['season']}):")
print(f"Standard xFP (4pt TD): {sample_qb['total_fantasy_points_exp']:.2f}")
print(f"Exp Pass TDs:          {sample_qb['pass_touchdown_exp']:.2f}")

# Find the matching row in our new df
new_val = final_xfp_df[
    (final_xfp_df['player_id'] == str(sample_qb['player_id'])) & 
    (final_xfp_df['season'] == sample_qb['season']) & 
    (final_xfp_df['week'] == sample_qb['week'])
]['expected_fantasy_points'].values[0]

print(f"Custom xFP (6pt TD):   {new_val:.2f}")
print(f"Difference:            +{new_val - sample_qb['total_fantasy_points_exp']:.2f} pts")

✅ Computed Custom xFP for 29,617 rows.


,player_id,season,week,expected_fantasy_points
0,00-0035228,2025,1.0,25.81
1,00-0037744,2025,1.0,15.43
2,00-0039376,2025,1.0,27.20
3,00-0037545,2025,1.0,14.13
4,0.0,2025,1.0,7.27



Sample Check (Josh Allen - Week 1.0, 2025):
Standard xFP (4pt TD): 43.82
Exp Pass TDs:          2.18
Custom xFP (6pt TD):   48.18
Difference:            +4.36 pts
